In [40]:
# pacotes
import numpy as np
import pandas as pd

In [41]:
# oculta mensagens de avisos
import warnings
warnings.filterwarnings("ignore")

In [42]:
# pd.set_option('display.max_columns', None)

In [144]:
# pd.set_option('display.max_rows', None)

In [44]:
# criando um range de datas úteis
business_dates = pd.bdate_range('2005-01-01', '2019-12-31')

In [45]:
# criando um dataframe que guardará essas informações
df = pd.DataFrame(
    index=business_dates, 
    columns=['Number of Negative Market Cap Stocks',
            'Tickers']
    )

In [46]:
for bdate in business_dates:
    try:
        # formatando a data
        day = str(bdate)[:4] + str(bdate)[5:7] + str(bdate)[8:10]

        # tratando o dataframe de fatores
        factors_path = f'../../../input/factors/{day}.csv'
        factors = pd.read_csv(factors_path, index_col=0)
        factors = factors[factors['TAQ_TICKER'] != '<undefined>']

        # tickers com market cap negativo 
        negative_mktcap = factors[factors['MARKETCAP'] < 0]['TAQ_TICKER']

        # tornando o objeto uma lista
        list_negative_factors = list(negative_mktcap)

        """
        preenchendo os dados
        coluna = 0    'Number of Negative Market Cap Stocks'
        coluna = 1    'Tickers with Negative Market Cap'
        """
        df.loc[bdate][0] = len(list_negative_factors)
        df.loc[bdate][1] = list_negative_factors
    except:
        pass

In [47]:
df = df.dropna(axis=0)

In [48]:
df

,Number of Negative Market Cap Stocks,Tickers
2005-01-03,99,"[BFEN, UBMT, BKSC, PLE, STIZ, MHG, PFDC, PZA, ..."
2005-01-04,127,"[AMCE, NHTB, BFEN, AABC, UBMT, CEBK, ELMS, HOS..."
2005-01-05,117,"[AABC, CEBK, BKSC, CCDC, HOST, EBIX, MBLA, HAR..."
2005-01-06,144,"[AMCE, NHTB, BFEN, QUIP, AABC, BKSC, JEN, CFNB..."
2005-01-07,142,"[DGSE, AMCE, ESBK, DMC, PATR, BFEN, QUIP, UBMT..."
...,...,...
2019-12-24,70,"[AIRT, OFED, BOTJ, SFBC, GYRO, HVBC, ICCH, KBL..."
2019-12-26,55,"[STND, SFBC, ICCH, KBLM, BPRN, NAOV, BRPA, LAC..."
2019-12-27,56,"[OFED, IROQ, AAMC, PIH, CNFR, GYRO, KBLM, WRLS..."
2019-12-30,48,"[HFBL, GYRO, ICCH, KBLM, NAOV, BRPA, MUDS, TIB..."


In [49]:
df.to_csv('../../../output/data/negative_mktvalue.csv', sep=',', encoding='utf-8')

Vamos analisar individualmente esses ticker que tiveram Market Cap negativo em pelo menos um dia.

In [50]:
all_tickers_negative_mktcap = []

for date in df.index:
    for i in range(len(df['Tickers'].loc[date])):
        all_tickers_negative_mktcap.append(df['Tickers'].loc[date][i])

In [51]:
# verificando o tamanho
len(all_tickers_negative_mktcap)

312963

In [52]:
# uma lista tem a função unique(), transformamos em um set (só ficam os elementos uma vez) e transformamos de volta em uma lista
all_tickers_negative_mktcap_unique = list(set(all_tickers_negative_mktcap))

In [53]:
# verificando o tamanho
len(all_tickers_negative_mktcap_unique)

2868

In [60]:
day = '20050103'
# tratando o dataframe de fatores
factors_path = f'../../../input/factors/{day}.csv'
factors = pd.read_csv(factors_path, index_col=0)
factors = factors[factors['TAQ_TICKER'] != '<undefined>']

In [73]:
permnos = []

for ticker in all_tickers_negative_mktcap_unique:
    try:
        permnos.append(factors[factors['TAQ_TICKER'] == ticker].index[0])
    except:
        pass

In [140]:
# dataframe que vai guardar todos as series das ações que tiveram market cap negativo
df_ = pd.DataFrame(index=business_dates)

In [145]:
for permno in permnos:
    try:
        # pegando dados para cada permno
        path = f'../../../../factors_database_/output/by_permno/{permno}.csv'
        temp = pd.read_csv(path, index_col=0)

        # nomearemos a coluna com o nome da ação
        col_name = temp['TAQ_TICKER'][0]
        # precisamos deixar os índices como datetime
        temp.index = pd.to_datetime(temp.index)

        # adicionando coluna ao dataframe
        df_[col_name] = temp['MARKETCAP']
    except:
        pass

In [150]:
# dropando linhas cujo todas colunas são NaN
df_ = df_.dropna(axis=0, how='all')

In [157]:
df_

,BDCO,RVP,OPOF,NTII,LDIS,OLGR,NOVT,BLMT,MFSF,ZONS,...,FAVS,SYNX,AFBK,GTIM,GVBK,FSCR,AMNT,PVSW,DITC,MBRG
2005-01-03,6946.368,106729.2000,138362.580,121201.92,294273.00,16949.320,26299.35,67757.34,113509.88,88477.400,...,30706.20,18390.460,-18011.250,7300.50,52165.60,44915.00,62808.85,107006.40,505944.600,142837.500
2005-01-04,6658.080,104409.0000,134870.400,117414.36,281661.30,17676.000,25482.60,68313.64,114875.20,83749.600,...,31364.19,17465.800,17611.000,7088.55,52615.76,45606.00,65038.75,105445.89,485982.160,142456.600
2005-01-05,6321.744,103944.9600,133947.180,116873.28,278578.44,17929.356,25645.95,67646.08,114169.00,79481.072,...,30706.20,17208.950,17851.150,7088.55,52668.72,45606.00,65410.40,99426.78,478410.200,140247.380
2005-01-06,6864.000,104873.0400,133911.054,115520.58,271291.68,17477.636,26136.00,67979.86,114121.92,75644.800,...,-30706.20,16284.290,18331.450,7182.75,52165.60,-46400.65,65782.05,104777.10,475622.342,140933.000
2005-01-07,6864.000,101160.7200,-137178.450,117684.90,251112.96,17676.000,24992.55,68758.68,112992.00,73078.280,...,-31071.75,14789.423,-18083.295,7182.75,-52271.52,46297.00,64741.43,102101.94,461545.380,-141294.855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,NaN,45264.6775,147443.640,NaN,NaN,NaN,NaN,NaN,338936.63,NaN,...,NaN,NaN,NaN,18184.45,325331.30,NaN,NaN,NaN,NaN,NaN
2019-12-26,NaN,47378.7500,145935.930,NaN,NaN,NaN,NaN,NaN,338680.70,NaN,...,NaN,NaN,NaN,18811.50,325636.06,NaN,NaN,NaN,NaN,NaN
2019-12-27,NaN,48032.2500,145675.980,NaN,NaN,NaN,NaN,NaN,337656.98,NaN,...,NaN,NaN,NaN,19313.14,326397.96,NaN,NaN,NaN,NaN,NaN
2019-12-30,NaN,50319.5000,140476.980,NaN,NaN,NaN,NaN,NaN,338766.01,NaN,...,NaN,NaN,NaN,19940.19,323426.55,NaN,NaN,NaN,NaN,NaN


In [152]:
df_.to_csv('../../../output/data/marketcap.csv', sep=',', encoding='utf-8')